In [0]:
%sql

CREATE OR REPLACE VIEW users.luis_herrera.field_manager_summary_view (
  snapshot_date,
  field_manager,
  sales_region,
  sales_subregion_level_1,
  sales_subregion_level_2,
  sales_subregion_level_3,
  sales_subregion_level_4,
  total_use_cases,
  total_estimated_monthly_dbu,
  high_confidence_count,
  high_confidence_pct,
  medium_confidence_count,
  medium_confidence_pct,
  low_confidence_count,
  low_confidence_pct,
  slip_usecase_count,
  slip_total_monthly_dbu,
  accel_usecase_count,
  accel_total_monthly_dbu,
  stale_usecase_count,
  stale_total_monthly_dbu,
  u3_stale_usecase_count,
  u3_stale_total_monthly_dbu,
  u5_stale_usecase_count,
  u5_stale_total_monthly_dbu,
  one_blocker_usecase_count,
  one_blocker_total_monthly_dbu,
  azure_medhigh_usecase_count,
  azure_medhigh_total_monthly_dbu,
  -- New fields for 3+ blockers
  threeplus_blocker_usecase_count,
  threeplus_blocker_total_monthly_dbu,
  top1_usecase_type,
  top1_usecase_type_count,
  top1_usecase_type_monthly_dbu,
  top2_usecase_type,
  top2_usecase_type_count,
  top2_usecase_type_monthly_dbu,
  top3_usecase_type,
  top3_usecase_type_count,
  top3_usecase_type_monthly_dbu,
  top1_slippage_category,
  top1_slip_usecase_count,
  top1_slippage_category_description,
  top2_slippage_category,
  top2_slip_usecase_count,
  top2_slippage_category_description,
  top3_slippage_category,
  top3_slip_usecase_count,
  top3_slippage_category_description,
  top1_accel_category,
  top1_accel_usecase_count,
  top1_accel_category_description,
  top2_accel_category,
  top2_accel_usecase_count,
  top2_accel_category_description,
  top3_accel_category,
  top3_accel_usecase_count,
  top3_accel_category_description,
  top1_vertical,
  top1_vertical_usecase_count,
  top2_vertical,
  top2_vertical_usecase_count,
  top3_vertical,
  top3_vertical_usecase_count,
  name,
  surname,
  generated_email,
  email,
  email_status
) AS
WITH field_manager_summary AS (
  SELECT
    any_value(snapshot_date) as snapshot_date,
    field_manager,
    any_value(sales_region) as sales_region,
    any_value(sales_subregion_level_1) as sales_subregion_level_1,
    any_value(sales_subregion_level_2) as sales_subregion_level_2,
    any_value(sales_subregion_level_3) as sales_subregion_level_3,
    any_value(sales_subregion_level_4) as sales_subregion_level_4,
    COUNT(*) AS total_use_cases,
    SUM(estimated_monthly_dollar_dbus) AS total_estimated_monthly_dbu,
    SUM(CASE WHEN confidence_level_normalized = 'High' THEN 1 ELSE 0 END) AS high_confidence_count,
    ROUND(100.0 * SUM(CASE WHEN confidence_level_normalized = 'High' THEN 1 ELSE 0 END) / COUNT(*), 2) AS high_confidence_pct,
    SUM(CASE WHEN confidence_level_normalized = 'Medium' THEN 1 ELSE 0 END) AS medium_confidence_count,
    ROUND(100.0 * SUM(CASE WHEN confidence_level_normalized = 'Medium' THEN 1 ELSE 0 END) / COUNT(*), 2) AS medium_confidence_pct,
    SUM(CASE WHEN confidence_level_normalized = 'Low' THEN 1 ELSE 0 END) AS low_confidence_count,
    ROUND(100.0 * SUM(CASE WHEN confidence_level_normalized = 'Low' THEN 1 ELSE 0 END) / COUNT(*), 2) AS low_confidence_pct,
    SUM(CASE WHEN likely_to_slip = TRUE THEN 1 ELSE 0 END) AS slip_usecase_count,
    SUM(CASE WHEN likely_to_slip = TRUE THEN estimated_monthly_dollar_dbus ELSE 0 END) AS slip_total_monthly_dbu,
    SUM(CASE WHEN can_be_accelerated = TRUE THEN 1 ELSE 0 END) AS accel_usecase_count,
    SUM(CASE WHEN can_be_accelerated = TRUE THEN estimated_monthly_dollar_dbus ELSE 0 END) AS accel_total_monthly_dbu,
    SUM(CASE WHEN stage IN ('U3', 'U5') AND last_modified_date < date_sub(current_date(), 14) THEN 1 ELSE 0 END) AS stale_usecase_count,
    SUM(CASE WHEN stage IN ('U3', 'U5') AND last_modified_date < date_sub(current_date(), 14) THEN estimated_monthly_dollar_dbus ELSE 0 END) AS stale_total_monthly_dbu,
    SUM(CASE WHEN stage = 'U3' AND last_modified_date < date_sub(current_date(), 14) THEN 1 ELSE 0 END) AS u3_stale_usecase_count,
    SUM(CASE WHEN stage = 'U3' AND last_modified_date < date_sub(current_date(), 14) THEN estimated_monthly_dollar_dbus ELSE 0 END) AS u3_stale_total_monthly_dbu,
    SUM(CASE WHEN stage = 'U5' AND last_modified_date < date_sub(current_date(), 14) THEN 1 ELSE 0 END) AS u5_stale_usecase_count,
    SUM(CASE WHEN stage = 'U5' AND last_modified_date < date_sub(current_date(), 14) THEN estimated_monthly_dollar_dbus ELSE 0 END) AS u5_stale_total_monthly_dbu,
    SUM(CASE WHEN num_of_blockers = 1 THEN 1 ELSE 0 END) AS one_blocker_usecase_count,
    SUM(CASE WHEN num_of_blockers = 1 THEN estimated_monthly_dollar_dbus ELSE 0 END) AS one_blocker_total_monthly_dbu,
    SUM(CASE WHEN stage IN ('U2', 'U4', 'U5') AND confidence_level_normalized IN ('Medium', 'High') AND lower(target_cloud) LIKE '%azure%' THEN 1 ELSE 0 END) AS azure_medhigh_usecase_count,
    SUM(CASE WHEN stage IN ('U2', 'U4', 'U5') AND confidence_level_normalized IN ('Medium', 'High') AND lower(target_cloud) LIKE '%azure%' THEN estimated_monthly_dollar_dbus ELSE 0 END) AS azure_medhigh_total_monthly_dbu,
    -- New fields for 3+ blockers
    SUM(CASE WHEN num_of_blockers >= 3 THEN 1 ELSE 0 END) AS threeplus_blocker_usecase_count,
    SUM(CASE WHEN num_of_blockers >= 3 THEN estimated_monthly_dollar_dbus ELSE 0 END) AS threeplus_blocker_total_monthly_dbu
  FROM users.luis_herrera.pipelineiq_view
  GROUP BY field_manager
),
slip_categories AS (
  SELECT
    field_manager,
    sc.slippage_category,
    COUNT(*) AS slip_usecase_count,
    ROW_NUMBER() OVER (PARTITION BY field_manager ORDER BY COUNT(*) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view sc
  WHERE likely_to_slip = TRUE
  GROUP BY field_manager, slippage_category
),
accel_categories AS (
  SELECT
    field_manager,
    ac.accel_category,
    COUNT(*) AS accel_usecase_count,
    ROW_NUMBER() OVER (PARTITION BY field_manager ORDER BY COUNT(*) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view ac
  WHERE can_be_accelerated = TRUE
  GROUP BY field_manager, accel_category
),
usecase_type_ranked AS (
  SELECT
    field_manager,
    usecase_type,
    COUNT(*) AS usecase_type_count,
    SUM(estimated_monthly_dollar_dbus) AS usecase_type_monthly_dbu,
    ROW_NUMBER() OVER (PARTITION BY field_manager ORDER BY SUM(estimated_monthly_dollar_dbus) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view
  GROUP BY field_manager, usecase_type
),
vertical_categories AS (
  SELECT
    field_manager,
    vertical,
    COUNT(*) AS vertical_usecase_count,
    ROW_NUMBER() OVER (PARTITION BY field_manager ORDER BY COUNT(*) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view
  GROUP BY field_manager, vertical
),
field_manager_emails AS (
  SELECT
    concat(trim(Name), ' ', trim(Surname)) AS field_manager,
    email
  FROM users.luis_herrera.field_managers_emails
)
SELECT
  fms.snapshot_date,
  fms.field_manager,
  fms.sales_region,
  fms.sales_subregion_level_1,
  fms.sales_subregion_level_2,
  fms.sales_subregion_level_3,
  fms.sales_subregion_level_4,
  fms.total_use_cases,
  fms.total_estimated_monthly_dbu,
  fms.high_confidence_count,
  fms.high_confidence_pct,
  fms.medium_confidence_count,
  fms.medium_confidence_pct,
  fms.low_confidence_count,
  fms.low_confidence_pct,
  fms.slip_usecase_count,
  fms.slip_total_monthly_dbu,
  fms.accel_usecase_count,
  fms.accel_total_monthly_dbu,
  fms.stale_usecase_count,
  fms.stale_total_monthly_dbu,
  fms.u3_stale_usecase_count,
  fms.u3_stale_total_monthly_dbu,
  fms.u5_stale_usecase_count,
  fms.u5_stale_total_monthly_dbu,
  fms.one_blocker_usecase_count,
  fms.one_blocker_total_monthly_dbu,
  fms.azure_medhigh_usecase_count,
  fms.azure_medhigh_total_monthly_dbu,
  -- New fields for 3+ blockers
  fms.threeplus_blocker_usecase_count,
  fms.threeplus_blocker_total_monthly_dbu,
  coalesce(u1.usecase_type, '') AS top1_usecase_type,
  coalesce(u1.usecase_type_count, 0) AS top1_usecase_type_count,
  coalesce(u1.usecase_type_monthly_dbu, 0) AS top1_usecase_type_monthly_dbu,
  coalesce(u2.usecase_type, '') AS top2_usecase_type,
  coalesce(u2.usecase_type_count, 0) AS top2_usecase_type_count,
  coalesce(u2.usecase_type_monthly_dbu, 0) AS top2_usecase_type_monthly_dbu,
  coalesce(u3.usecase_type, '') AS top3_usecase_type,
  coalesce(u3.usecase_type_count, 0) AS top3_usecase_type_count,
  coalesce(u3.usecase_type_monthly_dbu, 0) AS top3_usecase_type_monthly_dbu,
  coalesce(sc1.slippage_category, '') AS top1_slippage_category,
  coalesce(sc1.slip_usecase_count, 0) AS top1_slip_usecase_count,
  coalesce(sc1desc.description, '') AS top1_slippage_category_description,
  coalesce(sc2.slippage_category, '') AS top2_slippage_category,
  coalesce(sc2.slip_usecase_count, 0) AS top2_slip_usecase_count,
  coalesce(sc2desc.description, '') AS top2_slippage_category_description,
  coalesce(sc3.slippage_category, '') AS top3_slippage_category,
  coalesce(sc3.slip_usecase_count, 0) AS top3_slip_usecase_count,
  coalesce(sc3desc.description, '') AS top3_slippage_category_description,
  coalesce(ac1.accel_category, '') AS top1_accel_category,
  coalesce(ac1.accel_usecase_count, 0) AS top1_accel_usecase_count,
  coalesce(ac1desc.description, '') AS top1_accel_category_description,
  coalesce(ac2.accel_category, '') AS top2_accel_category,
  coalesce(ac2.accel_usecase_count, 0) AS top2_accel_usecase_count,
  coalesce(ac2desc.description, '') AS top2_accel_category_description,
  coalesce(ac3.accel_category, '') AS top3_accel_category,
  coalesce(ac3.accel_usecase_count, 0) AS top3_accel_usecase_count,
  coalesce(ac3desc.description, '') AS top3_accel_category_description,
  coalesce(v1.vertical, '') AS top1_vertical,
  coalesce(v1.vertical_usecase_count, 0) AS top1_vertical_usecase_count,
  coalesce(v2.vertical, '') AS top2_vertical,
  coalesce(v2.vertical_usecase_count, 0) AS top2_vertical_usecase_count,
  coalesce(v3.vertical, '') AS top3_vertical,
  coalesce(v3.vertical_usecase_count, 0) AS top3_vertical_usecase_count,
  -- New columns: name, surname, generated_email, email, email_status
  split(fms.field_manager, ' ')[0] AS name,
  split(fms.field_manager, ' ')[size(split(fms.field_manager, ' '))-1] AS surname,
  concat(
    lower(split(fms.field_manager, ' ')[0]), 
    '.', 
    lower(split(fms.field_manager, ' ')[size(split(fms.field_manager, ' '))-1]), 
    '@databricks.com'
  ) AS generated_email,
  CASE
    WHEN fme.email IS NULL THEN
      concat(
        lower(split(fms.field_manager, ' ')[0]), 
        '.', 
        lower(split(fms.field_manager, ' ')[size(split(fms.field_manager, ' '))-1]), 
        '@databricks.com'
      )
    ELSE fme.email
  END AS email,
  CASE
    WHEN fme.email IS NULL THEN 'correct'
    WHEN fme.email = concat(
      lower(split(fms.field_manager, ' ')[0]), 
      '.', 
      lower(split(fms.field_manager, ' ')[size(split(fms.field_manager, ' '))-1]), 
      '@databricks.com'
    ) THEN 'correct'
    ELSE 'mismatch'
  END AS email_status
FROM field_manager_summary fms
LEFT JOIN usecase_type_ranked u1
  ON fms.field_manager = u1.field_manager AND u1.rn = 1
LEFT JOIN usecase_type_ranked u2
  ON fms.field_manager = u2.field_manager AND u2.rn = 2
LEFT JOIN usecase_type_ranked u3
  ON fms.field_manager = u3.field_manager AND u3.rn = 3
LEFT JOIN slip_categories sc1
  ON fms.field_manager = sc1.field_manager AND sc1.rn = 1
LEFT JOIN slip_categories sc2
  ON fms.field_manager = sc2.field_manager AND sc2.rn = 2
LEFT JOIN slip_categories sc3
  ON fms.field_manager = sc3.field_manager AND sc3.rn = 3
LEFT JOIN users.luis_herrera.slippage_categories sc1desc
  ON sc1.slippage_category = sc1desc.type
LEFT JOIN users.luis_herrera.slippage_categories sc2desc
  ON sc2.slippage_category = sc2desc.type
LEFT JOIN users.luis_herrera.slippage_categories sc3desc
  ON sc3.slippage_category = sc3desc.type
LEFT JOIN accel_categories ac1
  ON fms.field_manager = ac1.field_manager AND ac1.rn = 1
LEFT JOIN accel_categories ac2
  ON fms.field_manager = ac2.field_manager AND ac2.rn = 2
LEFT JOIN accel_categories ac3
  ON fms.field_manager = ac3.field_manager AND ac3.rn = 3
LEFT JOIN users.luis_herrera.acceleration_categories ac1desc
  ON ac1.accel_category = ac1desc.type
LEFT JOIN users.luis_herrera.acceleration_categories ac2desc
  ON ac2.accel_category = ac2desc.type
LEFT JOIN users.luis_herrera.acceleration_categories ac3desc
  ON ac3.accel_category = ac3desc.type
LEFT JOIN vertical_categories v1
  ON fms.field_manager = v1.field_manager AND v1.rn = 1
LEFT JOIN vertical_categories v2
  ON fms.field_manager = v2.field_manager AND v2.rn = 2
LEFT JOIN vertical_categories v3
  ON fms.field_manager = v3.field_manager AND v3.rn = 3
LEFT JOIN field_manager_emails fme
  ON fms.field_manager = fme.field_manager
ORDER BY fms.field_manager;

In [0]:
%sql
SELECT * FROM users.luis_herrera.field_manager_summary_view;

In [0]:
%sql
CREATE OR REPLACE VIEW users.luis_herrera.sales_manager_summary_view (
  snapshot_date,
  sales_manager,
  sales_region,
  sales_subregion_level_1,
  sales_subregion_level_2,
  sales_subregion_level_3,
  sales_subregion_level_4,
  total_use_cases,
  total_estimated_monthly_dbu,
  high_confidence_count,
  high_confidence_pct,
  medium_confidence_count,
  medium_confidence_pct,
  low_confidence_count,
  low_confidence_pct,
  slip_usecase_count,
  slip_total_monthly_dbu,
  accel_usecase_count,
  accel_total_monthly_dbu,
  stale_usecase_count,
  stale_total_monthly_dbu,
  u3_stale_usecase_count,
  u3_stale_total_monthly_dbu,
  u5_stale_usecase_count,
  u5_stale_total_monthly_dbu,
  one_blocker_usecase_count,
  one_blocker_total_monthly_dbu,
  azure_medhigh_usecase_count,
  azure_medhigh_total_monthly_dbu,
  -- New fields for 3+ blockers
  threeplus_blocker_usecase_count,
  threeplus_blocker_total_monthly_dbu,
  top1_usecase_type,
  top1_usecase_type_count,
  top1_usecase_type_monthly_dbu,
  top2_usecase_type,
  top2_usecase_type_count,
  top2_usecase_type_monthly_dbu,
  top3_usecase_type,
  top3_usecase_type_count,
  top3_usecase_type_monthly_dbu,
  top1_slippage_category,
  top1_slip_usecase_count,
  top1_slippage_category_description,
  top2_slippage_category,
  top2_slip_usecase_count,
  top2_slippage_category_description,
  top3_slippage_category,
  top3_slip_usecase_count,
  top3_slippage_category_description,
  top1_accel_category,
  top1_accel_usecase_count,
  top1_accel_category_description,
  top2_accel_category,
  top2_accel_usecase_count,
  top2_accel_category_description,
  top3_accel_category,
  top3_accel_usecase_count,
  top3_accel_category_description,
  top1_vertical,
  top1_vertical_usecase_count,
  top2_vertical,
  top2_vertical_usecase_count,
  top3_vertical,
  top3_vertical_usecase_count,
  name,
  surname,
  email
) AS
WITH sales_manager_summary AS (
  SELECT
    any_value(snapshot_date) as snapshot_date,
    sales_manager,
    any_value(sales_region) as sales_region,
    any_value(sales_subregion_level_1) as sales_subregion_level_1,
    any_value(sales_subregion_level_2) as sales_subregion_level_2,
    any_value(sales_subregion_level_3) as sales_subregion_level_3,
    any_value(sales_subregion_level_4) as sales_subregion_level_4,
    COUNT(*) AS total_use_cases,
    SUM(estimated_monthly_dollar_dbus) AS total_estimated_monthly_dbu,
    SUM(CASE WHEN confidence_level_normalized = 'High' THEN 1 ELSE 0 END) AS high_confidence_count,
    ROUND(100.0 * SUM(CASE WHEN confidence_level_normalized = 'High' THEN 1 ELSE 0 END) / COUNT(*), 2) AS high_confidence_pct,
    SUM(CASE WHEN confidence_level_normalized = 'Medium' THEN 1 ELSE 0 END) AS medium_confidence_count,
    ROUND(100.0 * SUM(CASE WHEN confidence_level_normalized = 'Medium' THEN 1 ELSE 0 END) / COUNT(*), 2) AS medium_confidence_pct,
    SUM(CASE WHEN confidence_level_normalized = 'Low' THEN 1 ELSE 0 END) AS low_confidence_count,
    ROUND(100.0 * SUM(CASE WHEN confidence_level_normalized = 'Low' THEN 1 ELSE 0 END) / COUNT(*), 2) AS low_confidence_pct,
    SUM(CASE WHEN likely_to_slip = TRUE THEN 1 ELSE 0 END) AS slip_usecase_count,
    SUM(CASE WHEN likely_to_slip = TRUE THEN estimated_monthly_dollar_dbus ELSE 0 END) AS slip_total_monthly_dbu,
    SUM(CASE WHEN can_be_accelerated = TRUE THEN 1 ELSE 0 END) AS accel_usecase_count,
    SUM(CASE WHEN can_be_accelerated = TRUE THEN estimated_monthly_dollar_dbus ELSE 0 END) AS accel_total_monthly_dbu,
    SUM(CASE WHEN stage IN ('U3', 'U5') AND last_modified_date < date_sub(current_date(), 14) THEN 1 ELSE 0 END) AS stale_usecase_count,
    SUM(CASE WHEN stage IN ('U3', 'U5') AND last_modified_date < date_sub(current_date(), 14) THEN estimated_monthly_dollar_dbus ELSE 0 END) AS stale_total_monthly_dbu,
    SUM(CASE WHEN stage = 'U3' AND last_modified_date < date_sub(current_date(), 14) THEN 1 ELSE 0 END) AS u3_stale_usecase_count,
    SUM(CASE WHEN stage = 'U3' AND last_modified_date < date_sub(current_date(), 14) THEN estimated_monthly_dollar_dbus ELSE 0 END) AS u3_stale_total_monthly_dbu,
    SUM(CASE WHEN stage = 'U5' AND last_modified_date < date_sub(current_date(), 14) THEN 1 ELSE 0 END) AS u5_stale_usecase_count,
    SUM(CASE WHEN stage = 'U5' AND last_modified_date < date_sub(current_date(), 14) THEN estimated_monthly_dollar_dbus ELSE 0 END) AS u5_stale_total_monthly_dbu,
    SUM(CASE WHEN num_of_blockers = 1 THEN 1 ELSE 0 END) AS one_blocker_usecase_count,
    SUM(CASE WHEN num_of_blockers = 1 THEN estimated_monthly_dollar_dbus ELSE 0 END) AS one_blocker_total_monthly_dbu,
    SUM(CASE WHEN stage IN ('U2', 'U4', 'U5') AND confidence_level_normalized IN ('Medium', 'High') AND lower(target_cloud) LIKE '%azure%' THEN 1 ELSE 0 END) AS azure_medhigh_usecase_count,
    SUM(CASE WHEN stage IN ('U2', 'U4', 'U5') AND confidence_level_normalized IN ('Medium', 'High') AND lower(target_cloud) LIKE '%azure%' THEN estimated_monthly_dollar_dbus ELSE 0 END) AS azure_medhigh_total_monthly_dbu,
    -- New fields for 3+ blockers
    SUM(CASE WHEN num_of_blockers >= 3 THEN 1 ELSE 0 END) AS threeplus_blocker_usecase_count,
    SUM(CASE WHEN num_of_blockers >= 3 THEN estimated_monthly_dollar_dbus ELSE 0 END) AS threeplus_blocker_total_monthly_dbu
  FROM users.luis_herrera.pipelineiq_view
  GROUP BY sales_manager
),
slip_categories AS (
  SELECT
    sales_manager,
    sc.slippage_category,
    COUNT(*) AS slip_usecase_count,
    ROW_NUMBER() OVER (PARTITION BY sales_manager ORDER BY COUNT(*) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view sc
  WHERE likely_to_slip = TRUE
  GROUP BY sales_manager, slippage_category
),
accel_categories AS (
  SELECT
    sales_manager,
    ac.accel_category,
    COUNT(*) AS accel_usecase_count,
    ROW_NUMBER() OVER (PARTITION BY sales_manager ORDER BY COUNT(*) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view ac
  WHERE can_be_accelerated = TRUE
  GROUP BY sales_manager, accel_category
),
usecase_type_ranked AS (
  SELECT
    sales_manager,
    usecase_type,
    COUNT(*) AS usecase_type_count,
    SUM(estimated_monthly_dollar_dbus) AS usecase_type_monthly_dbu,
    ROW_NUMBER() OVER (PARTITION BY sales_manager ORDER BY SUM(estimated_monthly_dollar_dbus) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view
  GROUP BY sales_manager, usecase_type
),
vertical_categories AS (
  SELECT
    sales_manager,
    vertical,
    COUNT(*) AS vertical_usecase_count,
    ROW_NUMBER() OVER (PARTITION BY sales_manager ORDER BY COUNT(*) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view
  GROUP BY sales_manager, vertical
)
SELECT
  sms.snapshot_date,
  sms.sales_manager,
  sms.sales_region,
  sms.sales_subregion_level_1,
  sms.sales_subregion_level_2,
  sms.sales_subregion_level_3,
  sms.sales_subregion_level_4,
  sms.total_use_cases,
  sms.total_estimated_monthly_dbu,
  sms.high_confidence_count,
  sms.high_confidence_pct,
  sms.medium_confidence_count,
  sms.medium_confidence_pct,
  sms.low_confidence_count,
  sms.low_confidence_pct,
  sms.slip_usecase_count,
  sms.slip_total_monthly_dbu,
  sms.accel_usecase_count,
  sms.accel_total_monthly_dbu,
  sms.stale_usecase_count,
  sms.stale_total_monthly_dbu,
  sms.u3_stale_usecase_count,
  sms.u3_stale_total_monthly_dbu,
  sms.u5_stale_usecase_count,
  sms.u5_stale_total_monthly_dbu,
  sms.one_blocker_usecase_count,
  sms.one_blocker_total_monthly_dbu,
  sms.azure_medhigh_usecase_count,
  sms.azure_medhigh_total_monthly_dbu,
  -- New fields for 3+ blockers
  sms.threeplus_blocker_usecase_count,
  sms.threeplus_blocker_total_monthly_dbu,
  coalesce(u1.usecase_type, '') AS top1_usecase_type,
  coalesce(u1.usecase_type_count, 0) AS top1_usecase_type_count,
  coalesce(u1.usecase_type_monthly_dbu, 0) AS top1_usecase_type_monthly_dbu,
  coalesce(u2.usecase_type, '') AS top2_usecase_type,
  coalesce(u2.usecase_type_count, 0) AS top2_usecase_type_count,
  coalesce(u2.usecase_type_monthly_dbu, 0) AS top2_usecase_type_monthly_dbu,
  coalesce(u3.usecase_type, '') AS top3_usecase_type,
  coalesce(u3.usecase_type_count, 0) AS top3_usecase_type_count,
  coalesce(u3.usecase_type_monthly_dbu, 0) AS top3_usecase_type_monthly_dbu,
  coalesce(sc1.slippage_category, '') AS top1_slippage_category,
  coalesce(sc1.slip_usecase_count, 0) AS top1_slip_usecase_count,
  coalesce(sc1desc.description, '') AS top1_slippage_category_description,
  coalesce(sc2.slippage_category, '') AS top2_slippage_category,
  coalesce(sc2.slip_usecase_count, 0) AS top2_slip_usecase_count,
  coalesce(sc2desc.description, '') AS top2_slippage_category_description,
  coalesce(sc3.slippage_category, '') AS top3_slippage_category,
  coalesce(sc3.slip_usecase_count, 0) AS top3_slip_usecase_count,
  coalesce(sc3desc.description, '') AS top3_slippage_category_description,
  coalesce(ac1.accel_category, '') AS top1_accel_category,
  coalesce(ac1.accel_usecase_count, 0) AS top1_accel_usecase_count,
  coalesce(ac1desc.description, '') AS top1_accel_category_description,
  coalesce(ac2.accel_category, '') AS top2_accel_category,
  coalesce(ac2.accel_usecase_count, 0) AS top2_accel_usecase_count,
  coalesce(ac2desc.description, '') AS top2_accel_category_description,
  coalesce(ac3.accel_category, '') AS top3_accel_category,
  coalesce(ac3.accel_usecase_count, 0) AS top3_accel_usecase_count,
  coalesce(ac3desc.description, '') AS top3_accel_category_description,
  coalesce(v1.vertical, '') AS top1_vertical,
  coalesce(v1.vertical_usecase_count, 0) AS top1_vertical_usecase_count,
  coalesce(v2.vertical, '') AS top2_vertical,
  coalesce(v2.vertical_usecase_count, 0) AS top2_vertical_usecase_count,
  coalesce(v3.vertical, '') AS top3_vertical,
  coalesce(v3.vertical_usecase_count, 0) AS top3_vertical_usecase_count,
  -- New columns: name, surname, email
  split(sms.sales_manager, ' ')[0] AS name,
  split(sms.sales_manager, ' ')[size(split(sms.sales_manager, ' '))-1] AS surname,
  concat(
    lower(split(sms.sales_manager, ' ')[0]), 
    '.', 
    lower(split(sms.sales_manager, ' ')[size(split(sms.sales_manager, ' '))-1]), 
    '@databricks.com'
  ) AS email
FROM sales_manager_summary sms
LEFT JOIN usecase_type_ranked u1
  ON sms.sales_manager = u1.sales_manager AND u1.rn = 1
LEFT JOIN usecase_type_ranked u2
  ON sms.sales_manager = u2.sales_manager AND u2.rn = 2
LEFT JOIN usecase_type_ranked u3
  ON sms.sales_manager = u3.sales_manager AND u3.rn = 3
LEFT JOIN slip_categories sc1
  ON sms.sales_manager = sc1.sales_manager AND sc1.rn = 1
LEFT JOIN slip_categories sc2
  ON sms.sales_manager = sc2.sales_manager AND sc2.rn = 2
LEFT JOIN slip_categories sc3
  ON sms.sales_manager = sc3.sales_manager AND sc3.rn = 3
LEFT JOIN users.luis_herrera.slippage_categories sc1desc
  ON sc1.slippage_category = sc1desc.type
LEFT JOIN users.luis_herrera.slippage_categories sc2desc
  ON sc2.slippage_category = sc2desc.type
LEFT JOIN users.luis_herrera.slippage_categories sc3desc
  ON sc3.slippage_category = sc3desc.type
LEFT JOIN accel_categories ac1
  ON sms.sales_manager = ac1.sales_manager AND ac1.rn = 1
LEFT JOIN accel_categories ac2
  ON sms.sales_manager = ac2.sales_manager AND ac2.rn = 2
LEFT JOIN accel_categories ac3
  ON sms.sales_manager = ac3.sales_manager AND ac3.rn = 3
LEFT JOIN users.luis_herrera.acceleration_categories ac1desc
  ON ac1.accel_category = ac1desc.type
LEFT JOIN users.luis_herrera.acceleration_categories ac2desc
  ON ac2.accel_category = ac2desc.type
LEFT JOIN users.luis_herrera.acceleration_categories ac3desc
  ON ac3.accel_category = ac3desc.type
LEFT JOIN vertical_categories v1
  ON sms.sales_manager = v1.sales_manager AND v1.rn = 1
LEFT JOIN vertical_categories v2
  ON sms.sales_manager = v2.sales_manager AND v2.rn = 2
LEFT JOIN vertical_categories v3
  ON sms.sales_manager = v3.sales_manager AND v3.rn = 3
ORDER BY sms.sales_manager;

In [0]:
%sql
SELECT * 
FROM users.luis_herrera.sales_manager_summary_view

In [0]:
%sql

CREATE OR REPLACE VIEW users.luis_herrera.sales_manager_summary_view (
  snapshot_date,
  sales_manager,
  sales_region,
  sales_subregion_level_1,
  sales_subregion_level_2,
  sales_subregion_level_3,
  sales_subregion_level_4,
  total_use_cases,
  total_estimated_monthly_dbu,
  high_confidence_count,
  high_confidence_pct,
  medium_confidence_count,
  medium_confidence_pct,
  low_confidence_count,
  low_confidence_pct,
  slip_usecase_count,
  slip_total_monthly_dbu,
  accel_usecase_count,
  accel_total_monthly_dbu,
  stale_usecase_count,
  stale_total_monthly_dbu,
  u3_stale_usecase_count,
  u3_stale_total_monthly_dbu,
  u5_stale_usecase_count,
  u5_stale_total_monthly_dbu,
  one_blocker_usecase_count,
  one_blocker_total_monthly_dbu,
  azure_medhigh_usecase_count,
  azure_medhigh_total_monthly_dbu,
  -- New fields for 3+ blockers
  threeplus_blocker_usecase_count,
  threeplus_blocker_total_monthly_dbu,
  top1_usecase_type,
  top1_usecase_type_count,
  top1_usecase_type_monthly_dbu,
  top2_usecase_type,
  top2_usecase_type_count,
  top2_usecase_type_monthly_dbu,
  top3_usecase_type,
  top3_usecase_type_count,
  top3_usecase_type_monthly_dbu,
  top1_slippage_category,
  top1_slip_usecase_count,
  top1_slippage_category_description,
  top2_slippage_category,
  top2_slip_usecase_count,
  top2_slippage_category_description,
  top3_slippage_category,
  top3_slip_usecase_count,
  top3_slippage_category_description,
  top1_accel_category,
  top1_accel_usecase_count,
  top1_accel_category_description,
  top2_accel_category,
  top2_accel_usecase_count,
  top2_accel_category_description,
  top3_accel_category,
  top3_accel_usecase_count,
  top3_accel_category_description,
  top1_vertical,
  top1_vertical_usecase_count,
  top2_vertical,
  top2_vertical_usecase_count,
  top3_vertical,
  top3_vertical_usecase_count,
  name,
  surname,
  generated_email,
  email,
  email_status
) AS
WITH sales_manager_summary AS (
  SELECT
    any_value(snapshot_date) as snapshot_date,
    sales_manager,
    any_value(sales_region) as sales_region,
    any_value(sales_subregion_level_1) as sales_subregion_level_1,
    any_value(sales_subregion_level_2) as sales_subregion_level_2,
    any_value(sales_subregion_level_3) as sales_subregion_level_3,
    any_value(sales_subregion_level_4) as sales_subregion_level_4,
    COUNT(*) AS total_use_cases,
    SUM(estimated_monthly_dollar_dbus) AS total_estimated_monthly_dbu,
    SUM(CASE WHEN confidence_level_normalized = 'High' THEN 1 ELSE 0 END) AS high_confidence_count,
    ROUND(100.0 * SUM(CASE WHEN confidence_level_normalized = 'High' THEN 1 ELSE 0 END) / COUNT(*), 2) AS high_confidence_pct,
    SUM(CASE WHEN confidence_level_normalized = 'Medium' THEN 1 ELSE 0 END) AS medium_confidence_count,
    ROUND(100.0 * SUM(CASE WHEN confidence_level_normalized = 'Medium' THEN 1 ELSE 0 END) / COUNT(*), 2) AS medium_confidence_pct,
    SUM(CASE WHEN confidence_level_normalized = 'Low' THEN 1 ELSE 0 END) AS low_confidence_count,
    ROUND(100.0 * SUM(CASE WHEN confidence_level_normalized = 'Low' THEN 1 ELSE 0 END) / COUNT(*), 2) AS low_confidence_pct,
    SUM(CASE WHEN likely_to_slip = TRUE THEN 1 ELSE 0 END) AS slip_usecase_count,
    SUM(CASE WHEN likely_to_slip = TRUE THEN estimated_monthly_dollar_dbus ELSE 0 END) AS slip_total_monthly_dbu,
    SUM(CASE WHEN can_be_accelerated = TRUE THEN 1 ELSE 0 END) AS accel_usecase_count,
    SUM(CASE WHEN can_be_accelerated = TRUE THEN estimated_monthly_dollar_dbus ELSE 0 END) AS accel_total_monthly_dbu,
    SUM(CASE WHEN stage IN ('U3', 'U5') AND last_modified_date < date_sub(current_date(), 14) THEN 1 ELSE 0 END) AS stale_usecase_count,
    SUM(CASE WHEN stage IN ('U3', 'U5') AND last_modified_date < date_sub(current_date(), 14) THEN estimated_monthly_dollar_dbus ELSE 0 END) AS stale_total_monthly_dbu,
    SUM(CASE WHEN stage = 'U3' AND last_modified_date < date_sub(current_date(), 14) THEN 1 ELSE 0 END) AS u3_stale_usecase_count,
    SUM(CASE WHEN stage = 'U3' AND last_modified_date < date_sub(current_date(), 14) THEN estimated_monthly_dollar_dbus ELSE 0 END) AS u3_stale_total_monthly_dbu,
    SUM(CASE WHEN stage = 'U5' AND last_modified_date < date_sub(current_date(), 14) THEN 1 ELSE 0 END) AS u5_stale_usecase_count,
    SUM(CASE WHEN stage = 'U5' AND last_modified_date < date_sub(current_date(), 14) THEN estimated_monthly_dollar_dbus ELSE 0 END) AS u5_stale_total_monthly_dbu,
    SUM(CASE WHEN num_of_blockers = 1 THEN 1 ELSE 0 END) AS one_blocker_usecase_count,
    SUM(CASE WHEN num_of_blockers = 1 THEN estimated_monthly_dollar_dbus ELSE 0 END) AS one_blocker_total_monthly_dbu,
    SUM(CASE WHEN stage IN ('U2', 'U4', 'U5') AND confidence_level_normalized IN ('Medium', 'High') AND lower(target_cloud) LIKE '%azure%' THEN 1 ELSE 0 END) AS azure_medhigh_usecase_count,
    SUM(CASE WHEN stage IN ('U2', 'U4', 'U5') AND confidence_level_normalized IN ('Medium', 'High') AND lower(target_cloud) LIKE '%azure%' THEN estimated_monthly_dollar_dbus ELSE 0 END) AS azure_medhigh_total_monthly_dbu,
    -- New fields for 3+ blockers
    SUM(CASE WHEN num_of_blockers >= 3 THEN 1 ELSE 0 END) AS threeplus_blocker_usecase_count,
    SUM(CASE WHEN num_of_blockers >= 3 THEN estimated_monthly_dollar_dbus ELSE 0 END) AS threeplus_blocker_total_monthly_dbu
  FROM users.luis_herrera.pipelineiq_view
  GROUP BY sales_manager
),
slip_categories AS (
  SELECT
    sales_manager,
    sc.slippage_category,
    COUNT(*) AS slip_usecase_count,
    ROW_NUMBER() OVER (PARTITION BY sales_manager ORDER BY COUNT(*) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view sc
  WHERE likely_to_slip = TRUE
  GROUP BY sales_manager, slippage_category
),
accel_categories AS (
  SELECT
    sales_manager,
    ac.accel_category,
    COUNT(*) AS accel_usecase_count,
    ROW_NUMBER() OVER (PARTITION BY sales_manager ORDER BY COUNT(*) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view ac
  WHERE can_be_accelerated = TRUE
  GROUP BY sales_manager, accel_category
),
usecase_type_ranked AS (
  SELECT
    sales_manager,
    usecase_type,
    COUNT(*) AS usecase_type_count,
    SUM(estimated_monthly_dollar_dbus) AS usecase_type_monthly_dbu,
    ROW_NUMBER() OVER (PARTITION BY sales_manager ORDER BY SUM(estimated_monthly_dollar_dbus) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view
  GROUP BY sales_manager, usecase_type
),
vertical_categories AS (
  SELECT
    sales_manager,
    vertical,
    COUNT(*) AS vertical_usecase_count,
    ROW_NUMBER() OVER (PARTITION BY sales_manager ORDER BY COUNT(*) DESC) AS rn
  FROM users.luis_herrera.pipelineiq_view
  GROUP BY sales_manager, vertical
),
sales_manager_emails AS (
  SELECT
    concat(trim(Name), ' ', trim(Surname)) AS sales_manager,
    email
  FROM users.luis_herrera.sales_managers_emails
)
SELECT
  sms.snapshot_date,
  sms.sales_manager,
  sms.sales_region,
  sms.sales_subregion_level_1,
  sms.sales_subregion_level_2,
  sms.sales_subregion_level_3,
  sms.sales_subregion_level_4,
  sms.total_use_cases,
  sms.total_estimated_monthly_dbu,
  sms.high_confidence_count,
  sms.high_confidence_pct,
  sms.medium_confidence_count,
  sms.medium_confidence_pct,
  sms.low_confidence_count,
  sms.low_confidence_pct,
  sms.slip_usecase_count,
  sms.slip_total_monthly_dbu,
  sms.accel_usecase_count,
  sms.accel_total_monthly_dbu,
  sms.stale_usecase_count,
  sms.stale_total_monthly_dbu,
  sms.u3_stale_usecase_count,
  sms.u3_stale_total_monthly_dbu,
  sms.u5_stale_usecase_count,
  sms.u5_stale_total_monthly_dbu,
  sms.one_blocker_usecase_count,
  sms.one_blocker_total_monthly_dbu,
  sms.azure_medhigh_usecase_count,
  sms.azure_medhigh_total_monthly_dbu,
  -- New fields for 3+ blockers
  sms.threeplus_blocker_usecase_count,
  sms.threeplus_blocker_total_monthly_dbu,
  coalesce(u1.usecase_type, '') AS top1_usecase_type,
  coalesce(u1.usecase_type_count, 0) AS top1_usecase_type_count,
  coalesce(u1.usecase_type_monthly_dbu, 0) AS top1_usecase_type_monthly_dbu,
  coalesce(u2.usecase_type, '') AS top2_usecase_type,
  coalesce(u2.usecase_type_count, 0) AS top2_usecase_type_count,
  coalesce(u2.usecase_type_monthly_dbu, 0) AS top2_usecase_type_monthly_dbu,
  coalesce(u3.usecase_type, '') AS top3_usecase_type,
  coalesce(u3.usecase_type_count, 0) AS top3_usecase_type_count,
  coalesce(u3.usecase_type_monthly_dbu, 0) AS top3_usecase_type_monthly_dbu,
  coalesce(sc1.slippage_category, '') AS top1_slippage_category,
  coalesce(sc1.slip_usecase_count, 0) AS top1_slip_usecase_count,
  coalesce(sc1desc.description, '') AS top1_slippage_category_description,
  coalesce(sc2.slippage_category, '') AS top2_slippage_category,
  coalesce(sc2.slip_usecase_count, 0) AS top2_slip_usecase_count,
  coalesce(sc2desc.description, '') AS top2_slippage_category_description,
  coalesce(sc3.slippage_category, '') AS top3_slippage_category,
  coalesce(sc3.slip_usecase_count, 0) AS top3_slip_usecase_count,
  coalesce(sc3desc.description, '') AS top3_slippage_category_description,
  coalesce(ac1.accel_category, '') AS top1_accel_category,
  coalesce(ac1.accel_usecase_count, 0) AS top1_accel_usecase_count,
  coalesce(ac1desc.description, '') AS top1_accel_category_description,
  coalesce(ac2.accel_category, '') AS top2_accel_category,
  coalesce(ac2.accel_usecase_count, 0) AS top2_accel_usecase_count,
  coalesce(ac2desc.description, '') AS top2_accel_category_description,
  coalesce(ac3.accel_category, '') AS top3_accel_category,
  coalesce(ac3.accel_usecase_count, 0) AS top3_accel_usecase_count,
  coalesce(ac3desc.description, '') AS top3_accel_category_description,
  coalesce(v1.vertical, '') AS top1_vertical,
  coalesce(v1.vertical_usecase_count, 0) AS top1_vertical_usecase_count,
  coalesce(v2.vertical, '') AS top2_vertical,
  coalesce(v2.vertical_usecase_count, 0) AS top2_vertical_usecase_count,
  coalesce(v3.vertical, '') AS top3_vertical,
  coalesce(v3.vertical_usecase_count, 0) AS top3_vertical_usecase_count,
  -- New columns: name, surname, generated_email, email, email_status
  split(sms.sales_manager, ' ')[0] AS name,
  split(sms.sales_manager, ' ')[size(split(sms.sales_manager, ' '))-1] AS surname,
  concat(
    lower(split(sms.sales_manager, ' ')[0]), 
    '.', 
    lower(split(sms.sales_manager, ' ')[size(split(sms.sales_manager, ' '))-1]), 
    '@databricks.com'
  ) AS generated_email,
  CASE
    WHEN sme.email IS NULL THEN
      concat(
        lower(split(sms.sales_manager, ' ')[0]), 
        '.', 
        lower(split(sms.sales_manager, ' ')[size(split(sms.sales_manager, ' '))-1]), 
        '@databricks.com'
      )
    ELSE sme.email
  END AS email,
  CASE
    WHEN sme.email IS NULL THEN 'correct'
    WHEN sme.email = concat(
      lower(split(sms.sales_manager, ' ')[0]), 
      '.', 
      lower(split(sms.sales_manager, ' ')[size(split(sms.sales_manager, ' '))-1]), 
      '@databricks.com'
    ) THEN 'correct'
    ELSE 'mismatch'
  END AS email_status
FROM sales_manager_summary sms
LEFT JOIN usecase_type_ranked u1
  ON sms.sales_manager = u1.sales_manager AND u1.rn = 1
LEFT JOIN usecase_type_ranked u2
  ON sms.sales_manager = u2.sales_manager AND u2.rn = 2
LEFT JOIN usecase_type_ranked u3
  ON sms.sales_manager = u3.sales_manager AND u3.rn = 3
LEFT JOIN slip_categories sc1
  ON sms.sales_manager = sc1.sales_manager AND sc1.rn = 1
LEFT JOIN slip_categories sc2
  ON sms.sales_manager = sc2.sales_manager AND sc2.rn = 2
LEFT JOIN slip_categories sc3
  ON sms.sales_manager = sc3.sales_manager AND sc3.rn = 3
LEFT JOIN users.luis_herrera.slippage_categories sc1desc
  ON sc1.slippage_category = sc1desc.type
LEFT JOIN users.luis_herrera.slippage_categories sc2desc
  ON sc2.slippage_category = sc2desc.type
LEFT JOIN users.luis_herrera.slippage_categories sc3desc
  ON sc3.slippage_category = sc3desc.type
LEFT JOIN accel_categories ac1
  ON sms.sales_manager = ac1.sales_manager AND ac1.rn = 1
LEFT JOIN accel_categories ac2
  ON sms.sales_manager = ac2.sales_manager AND ac2.rn = 2
LEFT JOIN accel_categories ac3
  ON sms.sales_manager = ac3.sales_manager AND ac3.rn = 3
LEFT JOIN users.luis_herrera.acceleration_categories ac1desc
  ON ac1.accel_category = ac1desc.type
LEFT JOIN users.luis_herrera.acceleration_categories ac2desc
  ON ac2.accel_category = ac2desc.type
LEFT JOIN users.luis_herrera.acceleration_categories ac3desc
  ON ac3.accel_category = ac3desc.type
LEFT JOIN vertical_categories v1
  ON sms.sales_manager = v1.sales_manager AND v1.rn = 1
LEFT JOIN vertical_categories v2
  ON sms.sales_manager = v2.sales_manager AND v2.rn = 2
LEFT JOIN vertical_categories v3
  ON sms.sales_manager = v3.sales_manager AND v3.rn = 3
LEFT JOIN sales_manager_emails sme
  ON sms.sales_manager = sme.sales_manager
ORDER BY sms.sales_manager;

In [0]:
%sql
SELECT * 
FROM users.luis_herrera.sales_manager_summary_view